In [1]:
import os
import pandas as pd
import numpy as np
import json

In [2]:
read_dir='E:/文档/数据集/MathEC/'

save_dir='../data/math_ec/'

record_data=pd.read_csv(read_dir+'train_task_1_2.csv')
item_conc_data=pd.read_csv(read_dir+'question_metadata_task_1_2.csv')
conc_data=pd.read_csv(read_dir+'subject_metadata.csv')

prob_count_limit=15 #每个学生做过的习题下限

In [3]:
conc_data.columns=['knowledge_code','name','parent','level']

In [4]:
record=record_data.loc[:,['UserId','QuestionId','IsCorrect']]\
    .drop_duplicates(subset=['UserId','QuestionId'],keep='first').dropna(axis=0,how='any')

# 统计每个学生做了多少道题
problem_counter=record.groupby(by='UserId').count()

filtered_stu_boundary=prob_count_limit #学生做的题超过15道才算数

filtered_stu_id=problem_counter[problem_counter['QuestionId']>filtered_stu_boundary].index.to_numpy()

print('有效的学生数：',len(filtered_stu_id))

record=record.set_index('UserId').loc[filtered_stu_id,:].reset_index()
record.columns=['user_id','item_id','score']

有效的学生数： 118971


In [5]:
item_conc_data.columns=['item_id','knowledge_code']
item_conc_data=item_conc_data.set_index('item_id')
item_conc_data=item_conc_data[~item_conc_data.index.duplicated()]

In [6]:
item_unique=np.unique(record['item_id'])

In [7]:
iten_list,know_list=[],[]
for item in item_unique:
    item_knows=eval(item_conc_data.loc[item,'knowledge_code'])
    iten_list.extend([item]*len(item_knows))
    know_list.extend(item_knows)
item_open_df=pd.DataFrame({'item_id':iten_list,'knowledge_code':know_list})

In [8]:
know_unique=np.unique(item_open_df['knowledge_code'])

In [9]:
stu_unique=np.unique(record['user_id'])

In [10]:
stu_old_new=dict(zip(stu_unique,range(1,len(stu_unique)+1)))
item_old_new=dict(zip(item_unique,range(1,len(item_unique)+1)))
know_old_new=dict(zip(know_unique,range(1,len(know_unique)+1)))
know_id_level=dict(zip(conc_data['knowledge_code'],conc_data['level']))

In [11]:
with open(save_dir+'dict_knowledge_code.json','w') as f:
    dict_knowledge_code = {str(k):know_old_new[k] for k in  know_old_new.keys()}
    json.dump(dict_knowledge_code,f)

In [12]:
record['user_id']=record['user_id'].map(stu_old_new)
record['item_id']=record['item_id'].map(item_old_new)

item_open_df['item_id']=item_open_df['item_id'].map(item_old_new)
tmp=item_open_df['knowledge_code']
item_open_df['knowledge_code']=tmp.map(know_old_new)
item_open_df['level']=tmp.map(know_id_level)

conc_data['knowledge_code']=conc_data['knowledge_code'].map(know_old_new)
conc_data['parent']=conc_data['parent'].map(know_old_new)

conc_data.fillna(0)

print('Done')

Done


In [13]:
item_open_df=item_open_df.set_index('item_id')

item_df=pd.DataFrame(columns=['item_id','knowledge_code'],index=range(1,len(item_unique)+1))
for item in range(1,len(item_unique)+1):
    item_df.loc[item,'item_id']=item
    level=np.array(item_open_df.loc[item,['level']]).reshape(-1)
    low_level_idx=(level==level.max()) # 仅保留最低级别知识点
    item_df.loc[item,'knowledge_code']=np.array(item_open_df.loc[item,['knowledge_code']]).reshape(-1)[low_level_idx].tolist()

In [22]:
item_df.to_csv(save_dir+'item.csv',index=False)
record.to_csv(save_dir+'record.csv',index=False)

conc_data.to_csv(save_dir+'skill.csv',index=False)
conc_conc=conc_data.loc[:,['knowledge_code','parent']].dropna(how='any').astype('int')
conc_conc.to_csv(save_dir+'concept_relationship.csv',index=False)

In [23]:
print('学习者数：',len(stu_unique))
print('习题数：',len(item_unique))
print('知识点数：',len(know_unique))
print('记录数：',len(record))

学习者数： 118971
习题数： 27613
知识点数： 388
记录数： 15867850
